# PART 2

In [1]:
import mysql.connector as mysql
from haversine import haversine, Unit
from tabulate import tabulate

In [2]:
try:
    connection = mysql.connect(host="localhost", database="geolife", user="root", password="123", port=3306)
    cursor = connection.cursor()
except Exception as e:
    print("ERROR: Failed to connect to db:", e)

#### 1. How many users, activities, and trackpoints are there in the dataset (after it is inserted into the database).

In [3]:
cursor.execute( """
        SELECT 'User', COUNT(*) FROM User
        UNION ALL
        SELECT 'Activity', COUNT(*) FROM Activity
        UNION ALL
        SELECT 'TrackPoint', COUNT(*) FROM TrackPoint
    """)
rows = cursor.fetchall()
print(tabulate(rows, headers=['Table', 'Count'], tablefmt="pretty"))

+------------+---------+
|   Table    |  Count  |
+------------+---------+
|    User    |   182   |
|  Activity  |  16033  |
| TrackPoint | 9681069 |
+------------+---------+


#### 2. Find the average number of activities per user.

In [4]:
cursor.execute("""
    SELECT AVG(activity_count)
    FROM (
        SELECT COUNT(*) AS activity_count
        FROM Activity
        GROUP BY user_id
    ) AS activity_counts
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['Average Activities Per User'], tablefmt="pretty"))

+-----------------------------+
| Average Activities Per User |
+-----------------------------+
|           93.2151           |
+-----------------------------+


#### 3. Find the top 20 users with the highest number of activities.

In [5]:
cursor.execute("""
    SELECT user_id, COUNT(*) AS activity_count
    FROM Activity
    GROUP BY user_id
    ORDER BY activity_count DESC
    LIMIT 20
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['User ID', 'Activity Count'], tablefmt="pretty"))

+---------+----------------+
| User ID | Activity Count |
+---------+----------------+
|   128   |      2102      |
|   153   |      1793      |
|   025   |      715       |
|   163   |      704       |
|   062   |      691       |
|   144   |      563       |
|   041   |      399       |
|   085   |      364       |
|   004   |      346       |
|   140   |      345       |
|   167   |      320       |
|   068   |      280       |
|   017   |      265       |
|   003   |      261       |
|   014   |      236       |
|   126   |      215       |
|   030   |      210       |
|   112   |      208       |
|   011   |      201       |
|   039   |      198       |
+---------+----------------+


#### 4. Find all users who have taken a taxi.

In [6]:
cursor.execute("""
    SELECT DISTINCT user_id
    FROM Activity
    WHERE transportation_mode = 'taxi'
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['User ID'], tablefmt="pretty"))


+---------+
| User ID |
+---------+
|   010   |
|   058   |
|   062   |
|   078   |
|   080   |
|   085   |
|   098   |
|   111   |
|   128   |
|   163   |
+---------+


#### 5. Find all types of transportation modes and count how many activities that are tagged with these transportation mode labels. Do not count the rows where the mode is null.

In [7]:
cursor.execute("""
    SELECT transportation_mode, COUNT(*) AS activity_count
    FROM Activity
    WHERE transportation_mode IS NOT NULL
    GROUP BY transportation_mode
    ORDER BY activity_count DESC
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['Transportation Mode', 'Activity Count'], tablefmt='pretty'))


+---------------------+----------------+
| Transportation Mode | Activity Count |
+---------------------+----------------+
|        walk         |      480       |
|         car         |      419       |
|        bike         |      263       |
|         bus         |      199       |
|       subway        |      133       |
|        taxi         |       37       |
|      airplane       |       3        |
|        train        |       2        |
|         run         |       1        |
|        boat         |       1        |
+---------------------+----------------+


#### 6. a) Find the year with the most activities.

In [8]:
cursor.execute("""
    SELECT YEAR(start_date_time) as year, COUNT(*) AS activity_count
    FROM Activity
    GROUP BY year
    ORDER BY activity_count DESC
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['Year', 'Activity Count'], tablefmt='pretty'))
print("\n", rows[0][0], "is the year with the most activities.")


+------+----------------+
| Year | Activity Count |
+------+----------------+
| 2008 |      5888      |
| 2009 |      5879      |
| 2010 |      1487      |
| 2011 |      1204      |
| 2007 |      986       |
| 2012 |      588       |
| 2000 |       1        |
+------+----------------+

 2008 is the year with the most activities.


#### 6. b) Is this also the year with most recorded hours? 

In [9]:
cursor.execute("""
    SELECT YEAR(start_date_time) as year, 
        ROUND(SUM(TIMESTAMPDIFF(SECOND, start_date_time, end_date_time))/3600, 0) as total_hours
    FROM Activity
    GROUP BY year
    ORDER BY total_hours DESC
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['Year', 'Total Hours'], tablefmt='pretty'))
print("\n", rows[0][0], "is the year with the most hours.")

+------+-------------+
| Year | Total Hours |
+------+-------------+
| 2009 |    11612    |
| 2008 |    9180     |
| 2007 |    2282     |
| 2010 |    1389     |
| 2011 |    1132     |
| 2012 |     711     |
| 2000 |      0      |
+------+-------------+

 2009 is the year with the most hours.


#### 7. Find the total distance (in km) walked in 2008, by user with id=112.

In [10]:
cursor.execute("""
    SELECT tp.lat, tp.lon
    FROM TrackPoint tp
    JOIN Activity a ON tp.activity_id = a.id
    WHERE a.user_id = '112'
      AND a.transportation_mode = 'walk'
      AND YEAR(a.start_date_time) = 2008
    ORDER BY a.id, tp.date_time
""")
trackpoints = cursor.fetchall()

total_distance_km = 0
for i in range(1, len(trackpoints)):
    prev_point = (trackpoints[i-1][0], trackpoints[i-1][1])  # (lat, lon)
    curr_point = (trackpoints[i][0], trackpoints[i][1])      # (lat, lon)
    distance = haversine(prev_point, curr_point, unit=Unit.KILOMETERS)
    total_distance_km += distance

print(f"Total distance walked in 2008 by user 112: {total_distance_km:.2f} km")

Total distance walked in 2008 by user 112: 141.21 km


#### 8. Find the top 20 users who have gained the most altitude meters.

In [11]:
# Only remove altitude values which we KNOW are invalid, ie. they are -777 as specified it the assignment or NULL.
cursor.execute("""
SELECT
    a.user_id AS user_id,
    SUM(CASE WHEN tp2.altitude > tp1.altitude THEN tp2.altitude - tp1.altitude ELSE 0 END) AS total_elevation_gain
FROM Activity AS a 
JOIN TrackPoint AS tp1 ON a.id = tp1.activity_id
JOIN TrackPoint AS tp2 ON a.id = tp2.activity_id AND tp2.id = tp1.id + 1
WHERE tp1.altitude IS NOT NULL AND tp2.altitude IS NOT NULL AND tp1.altitude != -777 AND tp2.altitude != -777
GROUP BY user_id
ORDER BY total_elevation_gain DESC
LIMIT 20
""")

rows = cursor.fetchall()
print(tabulate(rows, headers=['User ID', 'Total Elevation Gain (feet)'], tablefmt='pretty'))

+---------+-----------------------------+
| User ID | Total Elevation Gain (feet) |
+---------+-----------------------------+
|   128   |           2135455           |
|   153   |           1820766           |
|   004   |           1089358           |
|   041   |           789890            |
|   003   |           766613            |
|   085   |           714049            |
|   163   |           673439            |
|   062   |           596103            |
|   144   |           588771            |
|   030   |           576377            |
|   039   |           481311            |
|   084   |           430319            |
|   000   |           398638            |
|   002   |           377503            |
|   167   |           370647            |
|   025   |           358098            |
|   037   |           325528            |
|   140   |           311151            |
|   126   |           272389            |
|   017   |           205270            |
+---------+-----------------------

#### 9. Find all users who have invalid activities, and the number of invalid activities per user

In [18]:
cursor.execute("""
    WITH InvalidActivities AS (
        SELECT
            tp1.activity_id
        FROM
            TrackPoint tp1
        JOIN
            TrackPoint tp2 ON tp1.activity_id = tp2.activity_id
                        AND tp2.id = tp1.id + 1
        WHERE
            TIMESTAMPDIFF(MINUTE, tp1.date_time, tp2.date_time) >= 5
        GROUP BY
            tp1.activity_id
    )
    SELECT  
        user_id,
        COUNT(activity_id) AS invalid_activities_count
    FROM
        InvalidActivities as ia
    JOIN Activity as a ON a.id = ia.activity_id
    GROUP BY
        user_id
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['User ID', 'Invalid Activity Count'], tablefmt='pretty'))

+---------+------------------------+
| User ID | Invalid Activity Count |
+---------+------------------------+
|   000   |          101           |
|   001   |           45           |
|   002   |           98           |
|   003   |          179           |
|   004   |          219           |
|   005   |           45           |
|   006   |           17           |
|   007   |           30           |
|   008   |           16           |
|   009   |           31           |
|   010   |           50           |
|   011   |           32           |
|   012   |           43           |
|   013   |           29           |
|   014   |          118           |
|   015   |           46           |
|   016   |           20           |
|   017   |          129           |
|   018   |           27           |
|   019   |           31           |
|   020   |           20           |
|   021   |           7            |
|   022   |           55           |
|   023   |           11           |
|

#### 10. Find the users who have tracked an activity in the Forbidden City of Beijing.  


In [24]:
cursor.execute("""
    SELECT user_id
    FROM Activity a
    JOIN TrackPoint tp ON a.id = tp.activity_id
    WHERE tp.lat BETWEEN 39.916 AND 39.917
      AND tp.lon BETWEEN 116.397 AND 116.398
    GROUP BY user_id
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['User ID'], tablefmt='pretty'))

+---------+
| User ID |
+---------+
|   018   |
|   019   |
+---------+


#### 11. Find all users who have registered transportation_mode and their most used transportation_mode. 

In [77]:
# The innermost subquery finds the user's most used transportation mode(s). The second subquery handles tie breaks alphabetically.
cursor.execute("""
    SELECT user_id, transportation_mode 
    FROM
        (SELECT user_id, transportation_mode, RANK() OVER(PARTITION BY user_id ORDER BY transportation_mode ASC) as lexrnk 
        FROM
            (SELECT user_id, transportation_mode, COUNT(*) AS activity_count,
            RANK() OVER(PARTITION BY user_id ORDER BY COUNT(*) DESC) as rnk
            FROM Activity
            WHERE transportation_mode IS NOT NULL
            GROUP BY user_id, transportation_mode) AS Ranked
        WHERE rnk = 1
        ) AS LexRanked
    WHERE lexrnk = 1
    ORDER BY user_id
    """)
rows = cursor.fetchall()
print(tabulate(rows, headers=['User ID',' Most Used Transportation Mode'], tablefmt='pretty'))

+---------+--------------------------------+
| User ID |  Most Used Transportation Mode |
+---------+--------------------------------+
|   010   |              taxi              |
|   020   |              bike              |
|   021   |              walk              |
|   052   |              bus               |
|   056   |              bike              |
|   058   |              car               |
|   060   |              walk              |
|   062   |              bus               |
|   064   |              bike              |
|   065   |              bike              |
|   067   |              walk              |
|   069   |              bike              |
|   073   |              walk              |
|   075   |              walk              |
|   076   |              car               |
|   078   |              walk              |
|   080   |              bike              |
|   081   |              bike              |
|   082   |              walk              |
|   084   